In [6]:
import argparse
import copy
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.utils import get_git_hash

from mmdet import __version__
from mmdet.apis import init_random_seed, set_random_seed, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.utils import collect_env, get_root_logger

import json
from cv2 import cv2
import matplotlib.pyplot as plt



parser = argparse.ArgumentParser(description='Train a detector')
parser.add_argument('--work-dir', help='the dir to save logs and models')
parser.add_argument(
    '--resume-from', help='the checkpoint file to resume from')
parser.add_argument(
    '--no-validate',
    action='store_true',
    help='whether not to evaluate the checkpoint during training')
group_gpus = parser.add_mutually_exclusive_group()
group_gpus.add_argument(
    '--gpus',
    type=int,
    help='number of gpus to use '
    '(only applicable to non-distributed training)')
group_gpus.add_argument(
    '--gpu-ids',
    type=int,
    nargs='+',
    help='ids of gpus to use '
    '(only applicable to non-distributed training)')
parser.add_argument('--seed', type=int, default=None, help='random seed')
parser.add_argument(
    '--deterministic',
    action='store_true',
    help='whether to set deterministic options for CUDNN backend.')
parser.add_argument(
    '--options',
    nargs='+',
    action=DictAction,
    help='override some settings in the used config, the key-value pair '
    'in xxx=yyy format will be merged into config file (deprecate), '
    'change to --cfg-options instead.')
parser.add_argument(
    '--cfg-options',
    nargs='+',
    action=DictAction,
    help='override some settings in the used config, the key-value pair '
    'in xxx=yyy format will be merged into config file. If the value to '
    'be overwritten is a list, it should be like key="[a,b]" or key=a,b '
    'It also allows nested list/tuple values, e.g. key="[(a,b),(c,d)]" '
    'Note that the quotation marks are necessary and that no white space '
    'is allowed.')
parser.add_argument(
    '--launcher',
    choices=['none', 'pytorch', 'slurm', 'mpi'],
    default='none',
    help='job launcher')
parser.add_argument('--local_rank', type=int, default=0)
args = parser.parse_args(args=[])

In [15]:
config = 'configs/balloon/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_balloon.py'

In [2]:
config = 'configs/balloon/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_balloon_dacon.py'

In [7]:
config = 'configs/balloon/faster_rcnn_r50_caffe_fpn_mstrain-poly_1x_balloon_dacon.py'

In [8]:
cfg = Config.fromfile(config)

In [12]:
cfg.data.test

{'type': 'CocoDataset',
 'ann_file': 'data/dacon/test_annotations.json',
 'img_prefix': 'data/dacon/test_image/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'MultiScaleFlipAug',
   'img_scale': (1333, 800),
   'flip': True,
   'transforms': [{'type': 'Resize', 'keep_ratio': True},
    {'type': 'RandomFlip'},
    {'type': 'Normalize',
     'mean': [123.675, 116.28, 103.53],
     'std': [58.395, 57.12, 57.375],
     'to_rgb': True},
    {'type': 'Pad', 'size_divisor': 32},
    {'type': 'ImageToTensor', 'keys': ['img']},
    {'type': 'Collect', 'keys': ['img']}]}],
 'classes': ('01_ulcer', '02_mass', '04_lymph', '05_bleeding')}

In [14]:
from glob import glob

In [18]:
sorted(glob('data/dacon/test_image/*'))

['data/dacon/test_image/test_200000.jpg',
 'data/dacon/test_image/test_200001.jpg',
 'data/dacon/test_image/test_200002.jpg',
 'data/dacon/test_image/test_200003.jpg',
 'data/dacon/test_image/test_200004.jpg',
 'data/dacon/test_image/test_200005.jpg',
 'data/dacon/test_image/test_200006.jpg',
 'data/dacon/test_image/test_200007.jpg',
 'data/dacon/test_image/test_200008.jpg',
 'data/dacon/test_image/test_200009.jpg',
 'data/dacon/test_image/test_200010.jpg',
 'data/dacon/test_image/test_200011.jpg',
 'data/dacon/test_image/test_200012.jpg',
 'data/dacon/test_image/test_200013.jpg',
 'data/dacon/test_image/test_200014.jpg',
 'data/dacon/test_image/test_200015.jpg',
 'data/dacon/test_image/test_200016.jpg',
 'data/dacon/test_image/test_200017.jpg',
 'data/dacon/test_image/test_200018.jpg',
 'data/dacon/test_image/test_200019.jpg',
 'data/dacon/test_image/test_200020.jpg',
 'data/dacon/test_image/test_200021.jpg',
 'data/dacon/test_image/test_200022.jpg',
 'data/dacon/test_image/test_20002

In [19]:
with open('data/dacon/test_annotations.json', 'r') as temp:
    output = json.load(temp)

In [21]:
output.keys()

dict_keys(['images', 'annotations', 'categories'])

In [ ]:

    # init distributed env first, since logger depends on the dist info.
    if args.launcher == 'none':
        distributed = False
    else:
        distributed = True
        init_dist(args.launcher, **cfg.dist_params)

    rank, _ = get_dist_info()
    # allows not to create
    if args.work_dir is not None and rank == 0:
        mmcv.mkdir_or_exist(osp.abspath(args.work_dir))
        timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
        json_file = osp.join(args.work_dir, f'eval_{timestamp}.json')

    # build the dataloader
    dataset = build_dataset(cfg.data.test)
    data_loader = build_dataloader(
        dataset,
        samples_per_gpu=samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=distributed,
        shuffle=False)

In [4]:
cfg.data.test

{'type': 'CocoDataset',
 'ann_file': 'data/coco/annotations/instances_val2017.json',
 'img_prefix': 'data/dacon/test_image/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'MultiScaleFlipAug',
   'img_scale': (1333, 800),
   'flip': True,
   'transforms': [{'type': 'Resize', 'keep_ratio': True},
    {'type': 'RandomFlip'},
    {'type': 'Normalize',
     'mean': [123.675, 116.28, 103.53],
     'std': [58.395, 57.12, 57.375],
     'to_rgb': True},
    {'type': 'Pad', 'size_divisor': 32},
    {'type': 'ImageToTensor', 'keys': ['img']},
    {'type': 'Collect', 'keys': ['img']}]}],
 'classes': ('01_ulcer', '02_mass', '04_lymph', '05_bleeding')}

In [4]:
if args.cfg_options is not None:
    cfg.merge_from_dict(args.cfg_options)
# import modules from string list.
if cfg.get('custom_imports', None):
    from mmcv.utils import import_modules_from_strings
    import_modules_from_strings(**cfg['custom_imports'])
# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

# work_dir is determined in this priority: CLI > segment in file > filename
if args.work_dir is not None:
    # update configs according to CLI args if args.work_dir is not None
    cfg.work_dir = args.work_dir
elif cfg.get('work_dir', None) is None:
    # use config filename as default work_dir if cfg.work_dir is None
    cfg.work_dir = osp.join('./work_dirs',
                            osp.splitext(osp.basename(config))[0])
if args.resume_from is not None:
    cfg.resume_from = args.resume_from
if args.gpu_ids is not None:
    cfg.gpu_ids = args.gpu_ids
else:
    cfg.gpu_ids = range(1) if args.gpus is None else range(args.gpus)

# init distributed env first, since logger depends on the dist info.
if args.launcher == 'none':
    distributed = False
else:
    distributed = True
    init_dist(args.launcher, **cfg.dist_params)
    # re-set gpu_ids with distributed training mode
    _, world_size = get_dist_info()
    cfg.gpu_ids = range(world_size)

In [5]:
# create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# dump config
cfg.dump(osp.join(cfg.work_dir, osp.basename(config)))
# init the logger before other steps
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

# init the meta dict to record some important information such as
# environment info and seed, which will be logged
meta = dict()
# log env info
env_info_dict = collect_env()
env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
dash_line = '-' * 60 + '\n'
logger.info('Environment info:\n' + dash_line + env_info + '\n' +
            dash_line)
meta['env_info'] = env_info
meta['config'] = cfg.pretty_text
# log some basic info
logger.info(f'Distributed training: {distributed}')
logger.info(f'Config:\n{cfg.pretty_text}')

# set random seeds
deterministic=False

seed = init_random_seed(777)
logger.info(f'Set random seed to {seed}, '
            f'deterministic: {deterministic}')
set_random_seed(seed, deterministic=deterministic)
cfg.seed = seed
meta['seed'] = seed
meta['exp_name'] = osp.basename(config)

2021-11-17 15:49:56,795 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.11 (default, Jul 27 2021, 14:32:16) [GCC 7.5.0]
CUDA available: False
GCC: gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0
PyTorch: 1.10.0+cu102
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=10.2, CUDNN_VERSION=7.6.5, CXX_COMPILER=/opt/rh/devtoolset-7/root/usr/bin/c++, CXX_FLAGS= -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -fopenmp -DNDEBUG -DUSE_KINETO -DUSE_FBGEMM -DUSE_QNNPACK -DUSE_PYTORCH_Q

2021-11-17 15:49:57,423 - mmdet - INFO - Set random seed to 777, deterministic: False


In [6]:
model = build_detector(
    cfg.model,
    train_cfg=cfg.get('train_cfg'),
    test_cfg=cfg.get('test_cfg'))
model.init_weights()

2021-11-17 15:49:57,765 - mmdet - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2021-11-17 15:49:57,767 - mmcv - INFO - load model from: torchvision://resnet50
2021-11-17 15:49:57,768 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2021-11-17 15:49:57,967 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2021-11-17 15:49:57,983 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2021-11-17 15:49:58,009 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2021-11-17 15:49:58,015 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'override': [{'name': 'shared_fcs

In [38]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!


In [39]:
datasets

[
 CocoDataset Train dataset with number of images 50098, and instance counts: 
 +--------------+-------+-------------+-------+--------------+-------+-----------------+-------+---------------+-------+
 | category     | count | category    | count | category     | count | category        | count | category      | count |
 +--------------+-------+-------------+-------+--------------+-------+-----------------+-------+---------------+-------+
 | 0 [01_ulcer] | 21925 | 1 [02_mass] | 6858  | 2 [04_lymph] | 8220  | 3 [05_bleeding] | 13095 | -1 background | 0     |
 +--------------+-------+-------------+-------+--------------+-------+-----------------+-------+---------------+-------+]

In [40]:
GroupSampler(datasets)

AssertionError: 

In [45]:
cfg.data.train['type']

'CocoDataset'

In [47]:
isinstance(cfg.data.train, (list, tuple))

False

In [48]:
from mmcv.utils import Registry, build_from_cfg

In [ ]:
build_from_cfg(cfg.data.train)

In [53]:
cfg.data.train

{'type': 'CocoDataset',
 'ann_file': 'data/dacon/train/train_annotations4.json',
 'img_prefix': 'data/dacon/train/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'LoadAnnotations', 'with_bbox': True},
  {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
  {'type': 'RandomFlip', 'flip_ratio': 0.5},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'Pad', 'size_divisor': 32},
  {'type': 'DefaultFormatBundle'},
  {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}],
 'classes': ('01_ulcer', '02_mass', '04_lymph', '05_bleeding')}

In [34]:
from torch.utils.data import Sampler


class GroupSampler(Sampler):

    def __init__(self, dataset, samples_per_gpu=1):
        assert hasattr(dataset, 'flag')
        self.dataset = dataset
        self.samples_per_gpu = samples_per_gpu
        self.flag = dataset.flag.astype(np.int64)
        self.group_sizes = np.bincount(self.flag)
        self.num_samples = 0
        for i, size in enumerate(self.group_sizes):
            self.num_samples += int(np.ceil(
                size / self.samples_per_gpu)) * self.samples_per_gpu

    def __iter__(self):
        indices = []
        for i, size in enumerate(self.group_sizes):
            if size == 0:
                continue
            indice = np.where(self.flag == i)[0]
            assert len(indice) == size
            np.random.shuffle(indice)
            num_extra = int(np.ceil(size / self.samples_per_gpu)
                            ) * self.samples_per_gpu - len(indice)
            indice = np.concatenate(
                [indice, np.random.choice(indice, num_extra)])
            indices.append(indice)
        indices = np.concatenate(indices)
        indices = [
            indices[i * self.samples_per_gpu:(i + 1) * self.samples_per_gpu]
            for i in np.random.permutation(
                range(len(indices) // self.samples_per_gpu))
        ]
        indices = np.concatenate(indices)
        indices = indices.astype(np.int64).tolist()
        assert len(indices) == self.num_samples
        return iter(indices)

    def __len__(self):
        return self.num_samples

In [17]:
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
if cfg.checkpoint_config is not None:
    # save mmdet version, config file content and class names in
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmdet_version=__version__ + get_git_hash()[:7],
        CLASSES=datasets[0].CLASSES)

In [13]:
# add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

NameError: name 'datasets' is not defined

In [22]:
train_detector(
    model,
    datasets,
    cfg,
    distributed=distributed,
    validate=(not args.no_validate),
    timestamp=timestamp,
    meta=meta)

RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW